In [1]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

## Steps to build pipeline

1. Tokenize the query
2. Tokenize the streams
3. Tokenize the stream

Run LSTM on query

In [2]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')
os.listdir('./')

['0-output',
 '2012_aurora_shooting.tsv.gz',
 '2012_pakistan_garment_factory_fires.tsv.gz',
 'hurricane_sandy.tsv.gz',
 'wisconsin_sikh_temple_shooting.tsv.gz']

In [3]:
df1 = pd.read_csv('./2012_aurora_shooting.tsv.gz', sep='\t')
df2 = pd.read_csv('./2012_pakistan_garment_factory_fires.tsv.gz', sep='\t')
df3 = pd.read_csv('./hurricane_sandy.tsv.gz', sep='\t')
df4 = pd.read_csv('./wisconsin_sikh_temple_shooting.tsv.gz', sep='\t')

In [4]:
# Title is the title of the article
print df1[df1['stream id']=='1342767285-e9b3e7dc6a9e31c8b2301848ab082117'].title[1]
# Each text is a stream (i.e., a sentence from the article)
print df1[df1['stream id']=='1342767285-e9b3e7dc6a9e31c8b2301848ab082117'].text[1]

Monument man to ride in Courage Classic for son - OutThereColorado.com
They 're cycling for those they care for , for those they know , and for those they nurture .


In [5]:
df1.head()

,hour,score,stream id,url,title,sentence,text
0,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,0,Jason Mezey -LRB- right -RRB- and fellow Coura...
1,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,1,"They 're cycling for those they care for , for..."
2,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,2,"For one cyclist , Monument resident Jason Meze..."
3,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,3,"The infant has one full left arm , half of his..."
4,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,4,"`` Dillon is a normally functioning , healthy ..."


In [6]:
df2.head()

,hour,score,stream id,url,title,sentence,text
0,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,0,Three men have been arrested on suspicion of a...
1,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,1,"The men - aged 26 , 21 and 18 - have been ques..."
2,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,2,Around 100 homes had to be evacuated because o...
3,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,3,No-one was hurt .
4,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,4,Forensic investigators from Greater Manchester...


In [7]:
df3.head()

,hour,score,stream id,url,title,sentence,text
0,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,0,hurricane Sandy as of 11:00 a.m. Wednesday .
1,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,1,Sandy is now a category one hurricane .
2,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,2,The National Hurricane Center reported Hurrica...
3,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,3,A hurricane warning is in effect for Jamaica a...
4,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,4,The government of the Bahamas has issued a hur...


In [8]:
df4.head()

,hour,score,stream id,url,title,sentence,text
0,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,0,A Sikh temple is harnessing the sun 's rays to...
1,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,1,"Leicester 's Guru Tegh Bahadur Gurdwara , in E..."
2,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,2,"The # 125,000 environmental project has been f..."
3,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,3,"Surinderpal Singh Rai , general secretary of t..."
4,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,4,He said : `` It 's a huge four-storey building...


In [9]:
os.listdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/trec-2013-data/')

['matches.tsv.gz',
 'nuggets.tsv.gz',
 'trec2013-ts-topics-test.xml',
 'updates.tsv.gz']

In [10]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/trec-2013-data/')

In [11]:
df5 = pd.read_csv('./nuggets.tsv.gz', sep='\t')
df6 = pd.read_csv('./matches.tsv.gz', sep='\t')
df7 = pd.read_csv('./updates.tsv.gz', sep='\t')

In [12]:
df5.head() # Nuggets

,query_id,nugget_id,timestamp,importance,nugget_len,nugget_text
0,TS13.1,VMTS13.01.050,1329945286,3,6,"train accident in Buenos Aires, Argentina."
1,TS13.1,VMTS13.01.051,1329945286,3,4,unknown number were killed
2,TS13.1,VMTS13.01.052,1329945286,3,2,Hundreds injured
3,TS13.1,VMTS13.01.053,1329946985,1,2,in 2012
4,TS13.1,VMTS13.01.054,1329947346,1,3,"February 22, 2012"


In [13]:
df6.head() # Matches

query_id    update_id   nugget_id  \
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0                     VMTS13.01.077   
       1329996913-06251bb5df849f4e5efe4245acb1c342-2                     VMTS13.01.050   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1                     VMTS13.01.050   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0                     VMTS13.01.078   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0                     VMTS13.01.052   

                                                      match_start  match_end  \
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0           35         53   
       1329996913-06251bb5df849f4e5efe4245acb1c342-2           50         74   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1           50        101   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0            0          8   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0            9         25   

                                                      auto_p  
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0       0  
       1329996913-06251bb5df849f4e5efe4245acb1c342-2       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0       0

In [14]:
df7.head() # Updates

,query_id,update_id,doc_id,sentence_id,update_len,duplicate_id,update_text
0,TS13.1,1329910380-3afda7882974e306bc75176f5ce37f3e-5,1329910380-3afda7882974e306bc75176f5ce37f3e,5,922,NaN,"The soldiers, Mr Massimiliano Latorre and Mr S..."
1,TS13.1,1329910549-b5765884364f2991653f58c01a85b6c7-1,1329910549-b5765884364f2991653f58c01a85b6c7,1,190,NaN,"“At the same time, the UM system Actors David ..."
2,TS13.1,1329911230-5599b4c78673045d4522f23f2a357509-7,1329911230-5599b4c78673045d4522f23f2a357509,7,24,NaN,They are protesting the English by vandalising...
3,TS13.1,1329911327-9bcecf79f6aee904fd6e962cad884b6b-23,1329911327-9bcecf79f6aee904fd6e962cad884b6b,23,14,NaN,But it was one of the biggest box office smash...
4,TS13.1,1329911633-579d7af8f8ae26a964b05b69ae375a85-24,1329911633-579d7af8f8ae26a964b05b69ae375a85,24,65,NaN,Lyle’ s article is worth reading in its entire...


In [ ]:
df7['query_id'].nunique()

In [ ]:
f = open('./trec2013-ts-topics-test.xml', 'rb')

out = f.readlines()
ox = BeautifulSoup(''.join(out),'lxml').contents[1]

In [ ]:
i.findAll('title')[0].text, ',',i.findAll('query')[0].text

In [ ]:
print "Wiki article, Query"
qs = []
c = 0
for i in ox.findAll('event'):
    qs.append(i.findAll('query')[0].text)
    print c, i.findAll('title')[0].text, ',',i.findAll('query')[0].text
    c+=1


# Only have ten events, so we should build the code to be more general

## This is the stream data for Aurora -- all the same query

In [ ]:
# This is the aurora shooting, query #2 in the xml
df1.head()

## these are the nuggets for all of the trek data

In [ ]:
df5.shape

In [ ]:
df5.head()

In [ ]:
df5[df5['nugget_id']=='VMTS13.01.077'] # Nuggets

Index in df6 (matches) is concatenation of (queryid, streamid, and sentence)

In [ ]:
df6.shape

In [ ]:
df6.head()

## These are the matches of the nuggets

In [ ]:
df7.head()

# This section tokenizes each dataset into a list of numeric indices

In [8]:
import os
import re
import sys
import pickle
import csv
import pandas as pd
from gensim import corpora
from collections import defaultdict    

def BuildIndexFiles(infile_list):
    """
    :type  infilename: str
    :param infilename: Fulll name of input file 

    :type  outfilename: str
    :param outfilename: Export filename, without the '.csv'
    """
    reload(sys)
    sys.setdefaultencoding('utf-8')
    all_tokens = []
    frequency = defaultdict(int)
    for idx, infilename in enumerate(infile_list):
        print('Loading %s %i of %i' % (infilename, idx, len(infile_list)) )

        df = pd.read_csv(infilename, sep='\t', encoding='latin-1')
        df['text'] = df['text'].str.replace('[^A-Za-z0-9]+', ' ').str.strip()
        texts = [ t.split(" ") for t in df['text'] ]

        for text in texts:
            for token in text:
                frequency[token] += 1
        texts = [ [token for token in text ]  for text in texts]
        # Collecting all the list of tokens
        all_tokens.append(texts)

    texts = sum(all_tokens, [])
    # Getting the dictionary with token info
    dictionary = corpora.Dictionary(texts)
    # Mapping to numeric list
    word2idx = dictionary.token2id
    dictionary.id2token = {v:k for k,v in dictionary.token2id.items()}
    idx2word = dictionary.id2token
    
    # Exporting the dictionaries
    print("Exporting word to index and dictionary to word indices")
    output = open('./0-output/LSTMDQN_Dic_token2id.pkl', 'ab+')
    pickle.dump(word2idx, output)
    output.close()

    output = open('./0-output/LSTMDQN_Dic_id2token.pkl', 'ab+')
    pickle.dump(idx2word, output)
    output.close()
    
    # Merging the dictionaries toa pandas data frame with summary info
    odf0 = pd.DataFrame.from_dict(dictionary.dfs, orient='index').reset_index()
    odf1 = pd.DataFrame.from_dict(word2idx, orient='index').reset_index()
    odf0.columns = ['id', 'frequency']
    odf1.columns = ['token', 'id']
    odf = pd.merge(left=odf0, right=odf1, on='id')
    odf = odf[['id','token', 'frequency']]
    odf.to_csv('./0-output/total_corpus_smry.csv')
    
    return dictionary
    
def TokenizeData(infile_list, outfile_list, word2idx):
    for idx, (infilename, outfilename) in enumerate(zip(infile_list, outfile_list)):
        df = pd.read_csv(infilename, sep='\t', encoding='latin-1')
        df['text'] = df['text'].str.replace('[^A-Za-z0-9]+', ' ').str.strip()
        texts = [ t.split(" ") for t in df['text'] ]

        frequency = defaultdict(int)
        for text in texts:
            for token in text:
                frequency[token] += 1
        texts = [ [token for token in text ]  for text in texts]
        
        text_numindex = [ [word2idx[i] for i in t] for t in texts]
        # Exporting files
        print('...file export to %s' % outfilename)
        with open(outfilename+'_numtext.csv', 'wb') as csvfile:
            data = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            data.writerow(['Text'])
            data.writerows(text_numindex)

        csvfile.close()
    print('...Exporting of tokenized data complete')

In [2]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')

infilelist = [
            './2012_aurora_shooting.tsv.gz', 
            './2012_pakistan_garment_factory_fires.tsv.gz',
            './hurricane_sandy.tsv.gz',
            './wisconsin_sikh_temple_shooting.tsv.gz'
]
outfilelist = [
            './0-output/2012_aurora_shooting',
            './0-output/2012_pakistan_garment_factory_fires',
            './0-output/hurricane_sandy',
            './0-output/wisconsin_sikh_temple_shooting'
    
]

In [3]:
mycorpora = BuildIndexFiles(infilelist)

In [9]:
TokenizeData(infilelist, outfilelist, word2idx=mycorpora.token2id)